In [1]:
import torch, argparse, os,random
from torch import nn
from dataloader import get_dataloaders_pretrain, get_dataloaders_gold_fine_tune, get_dataloaders_fine_tune
from model import Get_Bart_Model
from utils import SEED_EVERYTHING, AverageMeter, evaluate, bert_accuracy, top_k_accuracy
from tqdm import tqdm
from bnvocab import normalizeText
from transformers import AdamW
from transformers.optimization import get_linear_schedule_with_warmup
from nltk.translate.bleu_score import sentence_bleu

In [2]:
DEVICE = 'cpu'
DEVICE = 'cuda:3'

In [3]:
vs = 50000
# train_dataloader, valid_dataloader = get_dataloaders_pretrain(vocab_size = vs, batch_size = 1, prototype=2000)
train_dataloader, valid_dataloader = get_dataloaders_gold_fine_tune(vocab_size = vs, batch_size = 1, prototype=2000)
# train_dataloader, valid_dataloader = get_dataloaders_fine_tune(vocab_size = vs, batch_size = 1, prototype=2000)

In [4]:
bpe = train_dataloader.dataset.bpe
tokenizer = train_dataloader.dataset.tokenizer

In [5]:
model = Get_Bart_Model(train_dataloader.dataset.pad_token_id, device = DEVICE, vocab_size=vs)

In [6]:
os.listdir("../models/FINE_BART_Gold_afterSynth")

['current_state.pth',
 'bert_state_best_acc.pth',
 'bert_state_best_acc_topk.pth']

In [7]:
# state = torch.load("../models/BART_TEST_RUN/current_state.pth", map_location=DEVICE)
# state = torch.load("../models/FINE_BART_SYNT/current_state.pth", map_location=DEVICE)
# state = torch.load("../models/FINE_BART_ANNOTATED_Gold//current_state.pth", map_location=DEVICE)
# state = torch.load("../models/FINE_BART_SYNT_Gold/current_state.pth", map_location=DEVICE)
state = torch.load("../models/FINE_BART_Gold_afterSynth/current_state.pth", map_location=DEVICE)

model.load_state_dict(state['model_state_dict'])

<All keys matched successfully>

In [8]:
model.eval() 

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50000, 1024, padding_idx=207)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50000, 1024, padding_idx=207)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024, padding_idx=207)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, element

In [9]:
input_ids, attention_mask,decoder_input_ids, decoder_attention_mask, labels = next(iter(train_dataloader))
input_ids, attention_mask,decoder_input_ids, decoder_attention_mask, labels = (x.to(DEVICE) for x in [input_ids, attention_mask,decoder_input_ids, decoder_attention_mask, labels])
print("Masked = ", bpe.decode_ids(input_ids[0]))
print("Truth  = ",bpe.decode_ids(decoder_input_ids[0]))
outputs = model(input_ids=input_ids,attention_mask=attention_mask,decoder_input_ids=decoder_input_ids,
                                    decoder_attention_mask=decoder_attention_mask, labels = labels)
print(outputs.loss)
print("Pred   = ", bpe.decode_ids(outputs.logits.argmax(dim=-1)[0]))
print("Genrt1 = ", bpe.decode_ids(model.generate(input_ids, num_beams=1, min_length=1, max_length=50)[0]))
print("Genrt5 = ", bpe.decode_ids(model.generate(input_ids, num_beams=5, min_length=1, max_length=50)[0]))

Masked =  সাউথ বাংলা অ্যাগ্রিকালচার অ্যান্ড কমার্স ব্যাংকের ব্যবস্থাপনা পরিচালক (এমডি) গোলাম ফারুক বলেন, ‘বিটিআরসি তাদের টাকা ফেরত চেয়েছে, আমরা দিয়ে দেব।
Truth  =  সাউথ বাংলা অ্যাগ্রিকালচার অ্যান্ড কমার্স ব্যাংকের ব্যবস্থাপনা পরিচালক (এমডি) গোলাম ফারুক বলেন, ‘বিটিআরসি তাদের টাকা ফেরত চেয়েছে, আমরা দিয়ে দেবো।'
tensor(0.1763, device='cuda:3', grad_fn=<NllLossBackward>)
Pred   =  সাউথ বাংলা অ্যাগ্রিকালচার অ্যান্ড কমার্স ব্যাংকের ব্যবস্থাপনা পরিচালক (এমডি) গোলাম ফারুক বলেন, ‘বিটিআরসি তাদের টাকা ফেরত চেয়েছে, আমরা দিয়ে দেব'।'।
Genrt1 =  সাউথ বাংলা অ্যাগ্রিকালচার অ্যান্ড কমার্স ব্যাংকের ব্যবস্থাপনা পরিচালক (এমডি) গোলাম ফারুক বলেন, ‘বিটিআরসি তাদের টাকা ফেরত চেয়েছে, আমরা দিয়ে দেব'।
Genrt5 =   ⁇  সাউথ বাংলা অ্যাগ্রিকালচার অ্যান্ড কমার্স ব্যাংকের ব্যবস্থাপনা পরিচালক (এমডি) গোলাম ফারুক বলেন, ‘বিটিআরসি তাদের টাকা ফেরত চেয়েছে, আমরা দিয়ে দেব'।


In [48]:
print("Genrte = ", bpe.decode_ids(model.generate(input_ids, num_beams=5, min_length=1, max_length=256)[0]))

Genrte =   ⁇  অভিযোগপত্রে এ দুজনকে সাক্ষী হিসেবে রাখা হবে ।


In [131]:
bpe.encode("সত্যানুসন্ধিতসা")

['▁সত', '্যান', 'ুস', 'ন্ধ', 'িত', 'সা']

In [54]:
model.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50000, 1024, padding_idx=207)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50000, 1024, padding_idx=207)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024, padding_idx=207)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, element

## BLEU score

In [95]:
for input_ids, attention_mask,decoder_input_ids, decoder_attention_mask, labels in valid_dataloader:
    input_ids, attention_mask,decoder_input_ids, decoder_attention_mask, labels = (x.to(DEVICE) for x in [input_ids, attention_mask,decoder_input_ids, decoder_attention_mask, labels])
    summary_ids = model.generate(input_ids, num_beams=1, min_length=1, max_length=int(sent_input_ids.shape[1]*2))
    input_sent = bpe.decode_ids(input_ids[0])
    break

In [96]:
bpe.decode_ids(input_ids[0])

'উদ্ভিদ এবং প্রাণীজগৎ বা জীবনের অন্যান্য রূপগুলি, যেমনঃ ছত্রাককে সম্মিলিতভাবে জীবন (উদ্ভিদ বা প্রাণী কোনটিই নয়) হিসাবে উল্লেখ করা হয়।'

In [97]:
bpe.decode_ids(summary_ids[0][1:])

'উদ্ভিদ এবং প্রাণীজগৎ বা জীবনের অনন্য রূপগুলি, যেমনঃ ছত্রাককে সম্মিলিতভাবে জীবন (উদ্ভিদ'

In [98]:
bpe.decode_ids(labels[0])

'উদ্ভিদ এবং প্রাণীজগৎ বা জীবনের অন্যান্য রূপগুলি, যেমন : ছত্রাককে সম্মিলিতভাবে জীবন (উদ্ভিদ বা প্রাণী কোনোটিই নয়) হিসাবে উল্লেখ করা হয়।'

In [102]:
sentence_bleu([[1,2,3,4,5]], [23,4,5,3,22])

9.283142785759642e-155

In [16]:
S = []
sent = " মনে বড় আসা ।"
S.append(sent)
sent = " আসানুরুপ ফলাফম পেলাম না ।"
S.append(sent)
sent = "আমার কোন সত্যানুসন্ধিতসা নেই "
S.append(sent)
sent = "আমার সোনার বাং লা আমি তোমায় ভাল বাসা "
S.append(sent)
sent = "তবে কি শুধু আমি একাই এই কষ্ট বাচাব? "
S.append(sent)
sent = "আমার একটি বল এবং ব্যাট লাগবে। "
S.append(sent)
# sent = "ভাই, এখন পর্যন্ত মনে হচ্ছে আজ শেষ করা হয়তো সম্ভব হবে না, কারণ তন্বী আপু হয়তো সর্বোচ্চো এক-দুইশ এর বেশি ডিসিশন দিবে না তাই একার পক্ষে এতোগুলো শেষ করা যাবে না। তাই বিকল্প কিছু করা যা কিনা যেহেতু আজই শেষ হওয়া উচিত? Shahnewaz vai, kindly check "
# S.append(sent)
sent = "আমি গতকাল কাজটি "
S.append(sent)
sent = " আমি আমার ছেলেকে গতকা ল পড়ি । "
S.append(sent)
sent = " আমি আমার ছেলেকে গতকাল পর্যন্ত পড়ি ।"
S.append(sent)
sent = " এ ব্যপারে কিছু করা যা কিনা ভাবা দরকার। "
sent = 'ব্যপারে দরকার। কিনা কিছু এ করা যা ভাবা'
S.append(sent)
sent = "ভাই, এখন পর্যন্ত মনে হচ্ছে আজ শেষ করা হয়তো সম্ভব হবে না, কারণ তন্বী আপু হয়তো সর্বোচ্চো এক-দুইশ এর বেশি ডিসিশন দিবে না তাই একার পক্ষে এতোগুলো শেষ করা যাবে না। তাই বিকল্প কিছু করা যা কিনা যেহেতু আজই শেষ হওয়া উচিত? Shahnewaz vai, kindly check "
sent = " তাও কিনতে গিয়ে দেখি ৬০-১০০+ টাকা প্রায় বই গুলির দাম। "
S.append(sent)
sent = " তাও কিনতে গিয়ে দেখি ৬০-১০০+ টাকা প্রায় বইগুলো দাম। "
S.append(sent)
sent = " তের ভাইসমূহের সাথে আমার ভাল বন্ধুত্ব "
S.append(sent)

In [17]:
marker = ["*", "#"]
for i,s in enumerate(S):
    m = marker[i%2]
    s = " ".join(tokenizer(s))
    sent_input_ids = torch.tensor([bpe.encode_ids_with_bos_eos(s)]).long()
    sent_input_ids = sent_input_ids.to(DEVICE)
    summary_ids = model.generate(sent_input_ids, num_beams=1, min_length=1, max_length=int(sent_input_ids.shape[1]*1.2))
    print(m, s)
    print(m,bpe.decode_ids(summary_ids[0]))
#     print(len(str(i+1)))
#     print()

* মনে বড় আসা ।
* মনে বড় আসা ।
# আসানুরুপ ফলাফম পেলাম না ।
# আসুক ফলাফম পেলাম না ।
* আমার কোন সত্যানুসন্ধিতসা নেই
* আমার কোন সত্যানুসন্ধিতসা নেই।
# আমার সোনার বাং লা আমি তোমায় ভাল বাসা
# আমার সোনার বাংলা আমি তোমায় ভালোবাসি।
* তবে কি শুধু আমি একাই এই কষ্ট বাচাব ?
* তবে কি শুধু আমি একাই এই কষ্ট বাঁচাব?
# আমার একটি বল এবং ব্যাট লাগবে ।
# আমার একটি বল এবং ব্যাট লাগবে ।
* আমি গতকাল কাজটি
* আমি গতকাল কাজটি করেছি।
# আমি আমার ছেলেকে গতকা ল পড়ি ।
# আমি আমার ছেলেকে গতকাল পড়ি ।
* আমি আমার ছেলেকে গতকাল পর্যন্ত পড়ি ।
* আমি আমার ছেলেকে গতকাল পর্যন্ত পড়ি ।
# ব্যপারে দরকার । কিনা কিছু এ করা যা ভাবা
# ব্যপারে দরকার, কিনা কিছু এ করা যা ভাবা।
* তাও কিনতে গিয়ে দেখি ৬০ - ১০০+ টাকা প্রায় বই গুলির দাম ।
* তাও কিনতে গিয়ে দেখি 00 - 000+ টাকা প্রায় বই গুলির দাম ।
# তাও কিনতে গিয়ে দেখি ৬০ - ১০০+ টাকা প্রায় বইগুলো দাম ।
# তাও কিনতে গিয়ে দেখি 00 - 000+ টাকা প্রায় বইগুলো দাম ।
* তের ভাইসমূহের সাথে আমার ভাল বন্ধুত্ব
* তের ভাইসমূহের সাথে আমার ভাল বন্ধুত্ব।


In [75]:
for x in sent_input_ids[0]:
    print(bpe.words[x],end= " ")

<s> ▁তের ▁ভাই সমূহের ▁সাথে ▁আমার ▁ভালো ▁বন্ধুত্ব </s> 

In [76]:
for x in summary_ids[0]:
    print(bpe.words[x],end= " ")

<s> ▁তের ▁ভাইদের ▁সাথে ▁আমার ▁ভালো ▁বন্ধুত্ব </s> 

In [12]:
S = []
sent = "বাংলা ও বাঙালির গর্ব।বাঙালি প্রযোজক হিসেবে তিনি আরো বাণিজ্য-সফল বাংলা ছবি ও গান বানান, সাফল্যের শিখরে উঠুন। ওনার মত বাঙালি প্রযোজকদের হাত ধরে উঠে আসুক আরো রূপঙ্কর, রাঘবরা। বাংলা ছবি প্রযোজনায় সরকার, হাসান, রায়, মুখোপাধ্যায়, অধিকারীদের রাজ কায়েম হোক।"
S.append(sent)
sent = "চিনি না এনাকে। তবে হিন্দি সাম্রাজ্যবাদের বিরুদ্ধে প্রতিরোধ কিভাবে বাঙালিকে এই সূত্রে বাঁধে, বোঝা দরকার।"
S.append(sent)

In [13]:
for s in S:
    s = normalizeText(s)
    s = " ".join(tokenizer(s))
    sent_input_ids = torch.tensor([bpe.encode_ids_with_bos_eos(s)]).long()
    sent_input_ids = sent_input_ids.to(DEVICE)
    summary_ids = model.generate(sent_input_ids, num_beams=1, min_length=1, max_length=int(sent_input_ids.shape[1]*1.2))
    print("Original ", s)
    print("pred     ", bpe.decode_ids(summary_ids[0]))
    print()

Original  বাংলা ও বাঙালির গর্ব । বাঙালি প্রযোজক হিসেবে তিনি আরো বাণিজ্য - সফল বাংলা ছবি ও গান বানান , সাফল্যের শিখরে উঠুন । ওনার মত বাঙালি প্রযোজকদের হাত ধরে উঠে আসুক আরো রূপঙ্কর , রাঘবরা । বাংলা ছবি প্রযোজনায় সরকার , হাসান , রায় , মুখোপাধ্যায় , অধিকারীদের রাজ কায়েম হোক ।
pred      বাংলা ও বাঙালির গর্ব । বাঙালি প্রযোজক হিসেবে তিনি আরো বাণিজ্য - সফল বাংলা ছবি ও গান বানান , সাফল্যের শিখরে উঠুন । ওনার মতো বাঙালি প্রযোজকদের হাত ধরে উঠে আসুক আরো রূপঙ্কর , রাঘবরা । বাংলা ছবি প্রযোজনায় সরকার , হাসান , রায় , মুখোপাধ্যায় , অধিকারীদের রাজ কায়েম হোক ।

Original  চিনি না এনাকে । তবে হিন্দি সাম্রাজ্যবাদের বিরুদ্ধে প্রতিরোধ কিভাবে বাঙালিকে এই সূত্রে বাঁধে , বোঝা দরকার ।
pred      চিনি না এনাকে, তবে হিন্দি সাম্রাজ্যবাদের বিরুদ্ধে প্রতিরোধ কীভাবে বাঙালিকে এই সূত্রে বাঁধে, বোঝা দরকার ।



In [15]:
for s in S:
    s = " ".join(tokenizer(s))
    sent_input_ids = torch.tensor([bpe.encode_ids_with_bos_eos(s)]).long()
    sent_input_ids = sent_input_ids.to(DEVICE)
    summary_ids = model.generate(sent_input_ids, num_beams=1, min_length=1, max_length=int(sent_input_ids.shape[1]*1.2))
    print("Original ", s)
    print("pred ", bpe.decode_ids(summary_ids[0]))

Original  আমার কোন সত্যানুসন্ধিতসা নেই
pred  আমার কোনো সত্যানুসন্ধিতসা নেই।
Original  আমার সোনার বাংলা আমি তোমায় ভালবাসা
pred  আমার সোনার বাংলা আমি তোমায় ভালবাসা।
Original  তবে কি শুধু আমি একাই এই কষ্ট বাচাব ?
pred  তবে কি শুধু আমি একাই এই কষ্ট বাচাব?
Original  আমার একটি বল ও ব্যাট লাগবে ।
pred  আমার একটি বল ও ব্যাট লাগবে ।
Original  ভাই , এখন পর্যন্ত মনে হচ্ছে আজ শেষ করা হয়তো সম্ভব হবে না , কারণ তন্বী আপু হয়তো সর্বোচ্চো এক - দুইশ এর বেশি ডিসিশন দিবে না তাই একার পক্ষে এতোগুলো শেষ করা যাবে না । তাই বিকল্প কিছু করা যা কিনা যেহেতু আজই শেষ হওয়া উচিত ? Shahnewaz vai , kindly check
pred  ভাই, এখন পর্যন্ত মনে হচ্ছে আজ শেষ করা হয়তো সম্ভব হবে না , কারণ তন্বী আপু হয়তো সর্বোচ্চো এক - দুইশ এর বেশি ডিসিশন দিবে না তাই একার পক্ষে এতোগুলো শেষ করা যাবে না । তাই বিকল্প কিছু যা করা কিনা যেহেতু আজই শেষ হওয়া উচিত। (00?)।।
Original  আমি গতকাল কাজটি
pred  আমি গতকাল কাজটি করেছি।


In [91]:
sent_output_ids = model.generate(sent_input_ids, num_beams=10, min_length=1, max_length=int(sent_input_ids.shape[1]*2),num_return_sequences=10,top_p = 0.1,return_dict_in_generate=True)

In [92]:
for s in sent_output_ids['sequences']:
    print(bpe.decode_ids(s))

 ⁇  ভাইসমূহের সাথে আমার ভালো বন্ধুত্ব p
 ⁇  ভাইসমূহের সাথে আমার ভাল বন্ধুত্ব p
 ⁇  ভাইরা সাথে আমার ভালো বন্ধুত্ব p
 ⁇  ভাইদের সাথে আমার ভালো বন্ধুত্ব p p
 ⁇  ভাইসমূহের সাথে আমার ভালো বন্ধুত্ব ।
 ⁇ র ভাইসমূহের সাথে আমার ভালো বন্ধুত্ব
 ⁇  ভাইসমূহের সাথে আমার ভালো বন্ধু p
 ⁇  ভাইসমূহের সাথে আমার ভালো বন্ধুত্ব ?
 ⁇  ভাইদের সাথে আমার ভালো বন্ধুত্ব p
 ⁇  ভাইদেরকে সাথে আমার ভালো বন্ধুত্ব p


In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained("flax-community/bengali-t5-base")
tokenizer.encode("আমি বাংলার গান গাই")
tokenizer.decode([93, 1912, 814, 5995, 3, 1])


In [18]:
from bnlp import NLTKTokenizer
tokenizer = NLTKTokenizer()
from bpemb import BPEmb

In [19]:
bpe = BPEmb(lang="bn", vs= 1000)

downloading https://nlp.h-its.org/bpemb/bn/bn.wiki.bpe.vs1000.model


100%|██████████| 254809/254809 [00:01<00:00, 146982.37B/s]


downloading https://nlp.h-its.org/bpemb/bn/bn.wiki.bpe.vs1000.d100.w2v.bin.tar.gz


100%|██████████| 377006/377006 [00:02<00:00, 163679.91B/s]
